In [ ]:
# psycorg2, sqlite3, pymongo.MongoClient

# Чтение и запись данных. Часть 2

## 1. Работа с базой данных из pandas


### 1.1 

Подготовка данных:

In [1]:
import psycopg2
import pandas as pd
import sqlite3

pg_connection = {
    "host": "",
    "port": ,
    "dbname": "",
    "user": "",
    "password": ""
}
conn = psycopg2.connect(**pg_connection)

cursor = conn.cursor()

# получаем имена таблиц из базы
sql_str = "SELECT table_name FROM information_schema.tables WHERE table_schema='public';"
cursor.execute(sql_str)
tables_data = [a for a in cursor.fetchall()]
conn.commit()

print("Какие таблицы содержатся в Postgres: %s" % tables_data)

df = pd.read_sql_query("select * from public.ratings limit 5;", conn)
df.head()

Какие таблицы содержатся в Postgres: [('keywords',), ('links',), ('ratings',), ('course_purchases',)]


,userid,movieid,rating,timestamp
0,1,110,1.0,1425941529
1,1,147,4.5,1425942435
2,1,858,5.0,1425941523
3,1,1221,5.0,1425941546
4,1,1246,5.0,1425941556


**Задание простого уровня** <br>Посчитайте количество записей в каждой из таблиц **keywords, links, ratings** и выведите в формате имя *таблицы:количество строк*

*Результат работы:*
<pre>
table keywords:92838
table links:91686
table ratings:1555552
</pre>

In [2]:
import psycopg2

pg_connection = {
    "host": "",
    "port": ,
    "dbname": "",
    "user": "",
    "password": ""
}
conn = psycopg2.connect(**pg_connection)
cursor = conn.cursor()

table_names = [i[0] for i in tables_data]
str_tables = ''

for table in table_names:
        sql_str_tables = "SELECT count(*) FROM " + table
        cursor.execute(sql_str_tables)
        print(table + ':' + str(cursor.fetchall()[0][0]))
print('')
for table in table_names:        
        sql_str = "SELECT 1 FROM public.{} LIMIT 1;".format(table)
        cursor.execute(sql_str)
        row_count = [a for a in cursor.fetchall()][0][0]
        print("table {}:{}".format(table, row_count))
cursor.close()

keywords:92838
links:91686
ratings:1555552
course_purchases:154

table keywords:1
table links:1
table ratings:1
table course_purchases:1


### 1.2

**Задание среднего уровня** <br>Напечатайте колонки таблицы *ratings*. Подсказка: поисследуйте объект **cursor** с помощью встроенной в python команды *dir()*

In [6]:
import psycopg2

pg_connection = {
    "host": "",
    "port": ,
    "dbname": "",
    "user": "",
    "password": ""
}
conn = psycopg2.connect(**pg_connection)
cursor = conn.cursor()

sql_str = "SELECT * FROM ratings LIMIT 10;"
cursor.execute(sql_str)
conn.commit()

table_rating_fields = ''
for i in cursor.description:
    table_rating_fields += i[0] + '  '

# -- ВАШ КОД ТУТ --
#table_rating_fields = print(dir(cursor))
print("Поля таблицы ratings: %s" % table_rating_fields)

cursor.close()

Поля таблицы ratings: []


### 1.3

**Задание высокого уровня**
* Посмотрите на столбцы таблицы **links** и столбцы таблицы **ratings** по какому полю можно заджойнить эти две таблицы
* Посчитайте количество фильмов из links, у который нет оценок в ratings.

In [26]:
import psycopg2

pg_connection = {
    "host": "",
    "port": ,
    "dbname": "",
    "user": "",
    "password": ""
}
conn = psycopg2.connect(**pg_connection)
cursor = conn.cursor()

sql_str = "SELECT * FROM links LIMIT 10;"
cursor.execute(sql_str)
conn.commit()
for i in cursor.description:
    print(i[0])   #джойним по полю "movieid"

sql_str = "SELECT count(*) FROM links JOIN ratings ON ratings.movieid = links.movieid where ratings.rating IS NULL"
cursor.execute(sql_str)
print()
row_count = cursor.fetchall()
row_count = row_count[0]
conn.commit()

print("Количество фильмов без рейтингов: %s" % row_count)

cursor.close()

movieid
imdbid
tmdbid

Количество фильмов без рейтингов: 0


## 2. Работа с БД: MongoDB

### 2.1

Подготовка данных

In [4]:
from pymongo import MongoClient

mongo_connection = {
    "host": "",
    "port": ,
    "user": "",
    "password": "",
    "authSource": ""
}

mongo = MongoClient('mongodb://%s:%s@%s:%s/?authSource=%s' % (
    mongo_connection['user'], mongo_connection['password'],
    mongo_connection['host'], mongo_connection['port'], mongo_connection['authSource'])
)
db = mongo["movies"]

print("Коллекции, доступные в MongoDB: %s" % db.list_collection_names())

collection = db['tags']
print("Число документов в коллекции %s" % collection.estimated_document_count())

Коллекции, доступные в MongoDB: ['tags', 'users']
Число документов в коллекции 158680


**Задание простого уровня** <br>Посчитайте количество контента, у которого встречается тэг "toy"

* сделайте выборку такого контента (а точнее - поля `id`) в питоновский `list`
* подсчитайте количество элементов в `list`

In [5]:
selector = {'name': 'toy'}
exclude_id = {'_id': False}

mongo_cursor = collection.find(projection=exclude_id, filter={'name': 'toy'})

# -- ВАШ КОД ТУТ --

cursor_items = set([])
cursor_items = [item for item in mongo_cursor]

print("Количество контента с тэгом 'toy': %s\n" % len(cursor_items))

Количество контента с тэгом 'toy': 19



### 2.2

**Задание среднего уровня** <br> Выведите **только названия** 10 самых непопулярных тэгов <br>

Подсказка: посмотрите в уроке, как MongoDB позволяет также выполнять сложные агрегирующие запросы средствами СУБД

*Результат работы*
<pre>
Счётчик тэгов ['ancient tablet', 'remains', 'robespierre', 'social scandal', 'brain implant', 'adam west', 'arm cast', 'acab', 'gas explosion', 'female psychologist']
</pre>

In [6]:
# -- ВАШ КОД ТУТ --
pipeline = [
    {"$group":
        {"_id": "$name"}
    },
    {"$sort":
         {"tag_count": 1}
    },
    {"$limit": 10}
]

#for i in collection.aggregate(pipeline):
 #   print(i.get('_id'))

    
print([i.get('_id') for i in collection.aggregate(pipeline)])

['ancient tablet', 'remains', 'robespierre', 'social scandal', 'brain implant', 'adam west', 'arm cast', 'acab', 'gas explosion', 'female psychologist']


### 2.3

**Задание высокого уровня**

1. Подключитесь к Postgres. Найдите top-10 самого популярного контента
1. Сохраните в файл `content_popularity.json` в виде
<pre>
{'movieId': 931, 'popularity': 999}
{'movieId': 429, 'popularity': 111}
</pre>

Этот формат называется single-line json и его можно загрузить в MongoDB с помощью команды
<pre>
mongoimport --host $APP_MONGO_HOST --port $APP_MONGO_PORT --db movies --collection tags --file content_popularity.json
</pre>

**ВНИМАНИЕ!** Это просто пример как можно использовать single-line JSON, заливать его никуда не нужно - в задаче требуется только подготовить файл в правильном виде

*Подсказка* воспользуйтесь функцией `json.dumps`

*Результат работы* - файл `content_popularity.json`:

<pre>
{"movieId": 318, "popularity": 5626}
{"movieId": 356, "popularity": 5464}
{"movieId": 296, "popularity": 5242}
{"movieId": 593, "popularity": 5048}
{"movieId": 2571, "popularity": 4718}
{"movieId": 260, "popularity": 4556}
{"movieId": 480, "popularity": 4540}
{"movieId": 527, "popularity": 4120}
{"movieId": 1, "popularity": 4072}
{"movieId": 110, "popularity": 3928}
</pre>

In [7]:
# -- ВАШ КОД ТУТ --

import json

pg_connection = {
    "host": "",
    "port": ,
    "dbname": "",
    "user": "",
    "password": ""
}
conn = psycopg2.connect(**pg_connection)

cursor = conn.cursor()
sql_str = "SELECT movieid, count(ratings.rating) AS popularity FROM ratings GROUP BY ratings.movieid ORDER BY popularity DESC;"
cursor.execute(sql_str)
tables_data = [a for a in cursor.fetchall()]
conn.commit()

with open("content_popularity.json", "w") as write_file:
    write_file.write('<pre>')
    for i in tables_data:
        sql_dumper = json.dumps(['movieid: ', i[0], 'popularity: ', i[1]])
        write_file.write(sql_dumper)
    write_file.write('/pre')